# System Modeling and Dispatching

This notebook is intended to show a power system modeling framework that exploits the capabilities of Julia to improve performance and allow modelers to develop modular system to analyze problems with different complexities. 

The example system for this notebook is the [Small Test Systems for Power System Economic Studies](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=5589973), modified to include a PV-Plant and a Wind plant and time series of load and renewable energy. The system data can also be accessed in [Matpower format](http://www.pserc.cornell.edu/matpower/docs/ref/matpower5.0/case5.html). 

## 5 - Bus system example

It is possible to store the data for the 5-bus system described in the later sections in terms of the aforementioned type structure. 

The one-line diagram of the system is as follows, where the peak load of each bus is shown: 

![5_bus_system](5bus.png)

First the nodes and system parameters are defined in terms of the system and bus types. Which eventually allows for the re-use of the same information is some other components of the system change. 

## Intro
The objecitve is to exploit Julia's integration of dynamic types with the function dispatch. As explained in Julia's documentation: 

"Julia’s type system is dynamic, but gains some of the advantages of static type systems by making it possible to indicate that certain values are of specific types. This can be of great assistance in generating efficient code, but even more significantly, it allows method dispatch on the types of function arguments to be deeply integrated with the language."

The way the types are defined for PowerSystems is by using immutable types. There are two kinds of composite types (in 0.6) ``struct`` (an immutable type, old name was ``immutable``) and ``mutable struct`` (a mutable type, old name was ``type``). ``mutable struct``s are always allocated on the heap. ``struct``s will be allocated on the stack under certain conditions. (One case where they won't be allocated on the stack right now is if they have a field that is a ``mutable struct``). If a ``struct`` only has e.g. bitstype fields, the ``struct`` should always end up on the stack, and for example an array of such a ``struct`` should have a really nice dense memory layout.

For more details on Julia types, refer to the [documentation](https://docs.julialang.org/en/release-0.6/manual/types/)

## Packages

 - *[PowerSystems.jl](https://github.com/NREL/PowerSystems.jl)* We take advantage of Julia's dynamic types and functional dispatch in our implementation of PowerSystems to define data schemas for Power Systems Analysis problems.

 - *[PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)* We leverage the schemas defined in PowerSystems to create functions for defining Power Systems Analysis Problems. 

# Basic Economic Dispatch Example



## Packages Modeling and Result Inspection

In [303]:
# Modeling Packages
using PowerSystems;
using JuMP;
using Xpress; 
solver = Xpress.XpressSolver();

In [304]:
# Result Inspection Packages
using DataFrames;
using Plotly;
using Colors;

## Load the 5-Bus example

In [332]:
sys = include(expanduser("~/.julia/v0.6/PowerSystems/data_files/data_5bus.jl"))


PowerSystems.PowerSystem(PowerSystems.Bus[, , , , ], PowerSystems.Generator[, , , , , , ], PowerSystems.ElectricLoad[PowerSystems.StaticLoad("Bus2", true, , "P", 300.0, 98.61, 24x1 TimeSeries.TimeArray{Float64,1,DateTime,Array{Float64,1}} 2024-01-01T00:00:00 to 2024-01-01T23:00:00
│                     │        │
├─────────────────────┼────────┤
│ 2024-01-01T00:00:00 │ 0.7927 │
│ 2024-01-01T01:00:00 │ 0.7232 │
│ 2024-01-01T02:00:00 │ 0.711  │
│ 2024-01-01T03:00:00 │ 0.6777 │
│ 2024-01-01T04:00:00 │ 0.6682 │
│ 2024-01-01T05:00:00 │ 0.6717 │
│ 2024-01-01T06:00:00 │ 0.6876 │
│ 2024-01-01T07:00:00 │ 0.7118 │
│ 2024-01-01T08:00:00 │ 0.7563 │
│ 2024-01-01T09:00:00 │ 0.7984 │
│ 2024-01-01T10:00:00 │ 0.8278 │
│ 2024-01-01T11:00:00 │ 0.8404 │
   ⋮
│ 2024-01-01T13:00:00 │ 0.8346 │
│ 2024-01-01T14:00:00 │ 0.8229 │
│ 2024-01-01T15:00:00 │ 0.8169 │
│ 2024-01-01T16:00:00 │ 0.8241 │
│ 2024-01-01T17:00:00 │ 0.9057 │
│ 2024-01-01T18:00:00 │ 0.99   │
│ 2024-01-01T19:00:00 │ 1.0    │
│ 2024-01-01T20:00:0

Name: nodeA, Type: PVName: nodeB, Type: PQName: nodeC, Type: PVName: nodeD, Type: PVName: nodeE, Type: SFThermal Gen: 
   Name: Alta, Status: trueThermal Gen: 
   Name: Park City, Status: trueThermal Gen: 
   Name: Solitude, Status: trueThermal Gen: 
   Name: Sundance, Status: trueThermal Gen: 
   Name: Brighton, Status: trueRenewableFix: 
   Name: SolarBusC, Status: trueRenewableCurtailment: 
   Name: WindBusA, Status: trueName: nodeB, Type: PQName: nodeC, Type: PVName: nodeD, Type: PVName: nodeD, Type: PVName: nodeA, Type: PVName: nodeB, Type: PQName: nodeA, Type: PVName: nodeD, Type: PVName: nodeA, Type: PVName: nodeE, Type: SFName: nodeB, Type: PQName: nodeC, Type: PVName: nodeC, Type: PVName: nodeD, Type: PVName: nodeD, Type: PVName: nodeE, Type: SF

## Populate Economic Dispatch Problem
Create a function that populates an economic dispatch problem based on the data defined in the PowerSystems instance

## Modular function:

In [319]:
# Thermal Generator Operating Limits
function powerconstraints(m, Pg, Generator::ThermalGen)
    for var in Pg
        @constraint(m, var >= Generator.tech.realpowerlimits.min)
        @constraint(m, var <= Generator.tech.realpowerlimits.max)
    end
end

# RE Generator Operating Limits
function powerconstraints(m, Pg, Generator::RenewableGen)
    for (time, var) in enumerate(Pg)
        @constraint(m, var <= Generator.tech.installedcapacity*Generator.scalingfactor.values[time])
    end
end

function makeED(sys,solver)
    EconomicDispatch = Model(solver=solver)
    gSet = [sys.generators[i].name for i in 1:length(sys.generators)]
    @variable(EconomicDispatch, Pg[gSet, t = 1:sys.timesteps] >=0 )

    for (ix, name) in enumerate(Pg.indexsets[1])
        if name == sys.generators[ix].name
            powerconstraints(EconomicDispatch, Pg[name,:], sys.generators[ix])
            
        else
            error("Bus name in Array and variable do not match")
        end
    end
    
    # Power Balance
    @constraint(EconomicDispatch, Pbal[t = 1:sys.timesteps], sum(Pg[gen.name,t] for gen in sys.generators) == sum(ld.maxrealpower*ld.scalingfactor.values[t] for ld in sys.loads));
    
    # Objective
    @objective(EconomicDispatch, :Min, sum(sum(Pg[gen.name,i]*gen.econ.variablecost for gen in sys.generators if typeof(gen)==PowerSystems.ThermalGen) for i in 1:sys.timesteps)) ;

    return(EconomicDispatch)
end

makeED (generic function with 1 method)

## Populate an Economic Dispatch Problem using 5-Bus case

In [320]:
# populate an economic dispatch
EconomicDispatch = makeED(sys,solver)

Minimization problem with:
 * 312 linear constraints
 * 168 variables
Solver is Xpress

## Solve the problem and inspect results

In [335]:
# run economic dispatch
solve(EconomicDispatch)

:Optimal

In [336]:
# look at results
getobjectivevalue(EconomicDispatch)

240051.94834426

In [337]:
getobjbound(EconomicDispatch)

0.0

## Results are contained in the model

In [338]:
EconomicDispatch.objDict

Dict{Symbol,Any} with 2 entries:
  :Pg   => Pg[i,t] ≥ 0 ∀ i ∈ {Alta,Park City,…,SolarBusC,WindBusA}, t ∈ {1,2,…,…
  :Pbal => JuMP.ConstraintRef[Pg[Alta,1] + Pg[Park City,1] + Pg[Solitude,1] + P…

In [339]:
getvalue(EconomicDispatch.objDict[:Pg])

Pg: 2 dimensions:
[     Alta,:]
  [     Alta, 1] = 40.0
  [     Alta, 2] = 40.0
  [     Alta, 3] = 14.790807519999888
  [     Alta, 4] = 0.0
  [     Alta, 5] = 0.0
  [     Alta, 6] = 6.603711620000027
  [     Alta, 7] = 40.0
  [     Alta, 8] = 40.0
  [     Alta, 9] = 40.0
  [     Alta,10] = 40.0
  [     Alta,11] = 40.0
  [     Alta,12] = 40.0
  [     Alta,13] = 40.0
  [     Alta,14] = 40.0
  [     Alta,15] = 40.0
  [     Alta,16] = 40.0
  [     Alta,17] = 40.0
  [     Alta,18] = 40.0
  [     Alta,19] = 40.0
  [     Alta,20] = 40.0
  [     Alta,21] = 40.0
  [     Alta,22] = 40.0
  [     Alta,23] = 40.0
  [     Alta,24] = 40.0
[Park City,:]
  [Park City, 1] = 164.57114955999998
  [Park City, 2] = 0.14935081999995248
  [Park City, 3] = 0.0
  [Park City, 4] = 0.0
  [Park City, 5] = 0.0
  [Park City, 6] = 0.0
  [Park City, 7] = 0.5524044600000764
  [Park City, 8] = 46.11368349999994
  [Park City, 9] = 102.33231955999995
  [Park City,10] = 143.4532219399999
  [Park City,11] = 170.0
  [Park C

## Create a function to extract results into a useful structure

In [326]:
# A function to collect results
function getEDresult(EconomicDispatch,sys)
    res = getvalue(EconomicDispatch.objDict[:Pg]);
    res = DataFrame(Dict(gen.name=>res[gen.name,:] for gen in sys.generators));
    return(res)
end

getEDresult (generic function with 1 method)

In [340]:
getEDresult(EconomicDispatch,sys)

,Alta,Brighton,Park City,SolarBusC,Solitude,Sundance,WindBusA
1,40.0,600.0,164.571,0.0,0.0,0.0,118.225
2,40.0,600.0,0.149351,0.0,0.0,0.0,119.015
3,14.7908,600.0,0.0,0.0,0.0,0.0,119.718
4,0.0,586.458,0.0,0.0,0.0,0.0,120.0
5,0.0,594.601,0.0,0.0,0.0,0.0,119.84
6,6.60371,600.0,0.0,0.0,0.0,0.0,119.46
7,40.0,600.0,0.552404,0.0,0.0,0.0,119.09
8,40.0,600.0,46.1137,0.0,0.0,0.0,117.903
9,40.0,600.0,102.332,0.0,0.0,0.0,114.864
10,40.0,600.0,143.453,21.0663,0.0,0.0,111.318


## Make a simple dispatch plot

In [329]:
p = Plot(melt(getEDresult(EconomicDispatch,sys)),x=1:size(res)[1],y=:value,group=:variable,Layout(title="Generator Dispatch"))
plot(p)

## More interesting dispatch stack

In [330]:
# function to stack dispatch into area plot
function area2(result)
    function _stacked_area!(traces)
        for (i, tr) in enumerate(traces[2:end])
            for j in 1:min(length(traces[i]["y"]), length(tr["y"]))
                tr["y"][j] += traces[i]["y"][j]
            end
        end
        return(traces)
    end
    names = result.colindex.names
    colors = distinguishable_colors(length(names))
    traces = PlotlyBase.GenericTrace{Dict{Symbol,Any}}[]
    for i in 1:length(names)
        push!(traces,scatter(;name=names[i],x=1:sys.timesteps,y=result[names[i]],fill="tonexty", fillcolor = colors[i],mode="none"))
    end
 
    traces = _stacked_area!(traces)
    plot(traces, Layout(title="Dispatch Stack"))
    #return(traces)
end ;

In [331]:
area2(getEDresult(EconomicDispatch,sys))